# Actual Implementation

In [1]:
from haloflow.dann.data_loader import SimulationDataset
from haloflow.dann import model as M
from haloflow.dann import train as T
from haloflow.dann import evalutate as E
from haloflow.dann import visualise as V

from haloflow import config as C

In [2]:
import torch

In [26]:
# Configuration
config = {
    'sims': ['TNG50', 'TNG100', 'Eagle100', 'Simba100', 'TNG_ALL'],
    'obs': 'mags',
    'dat_dir': C.get_dat_dir(),
    'input_dim': None,  # Will be inferred from data
    'num_domains': 4,
    'batch_size': 128,
    'num_epochs': 100,
    'lr': 0.001,
    'alpha': 0.5,
}

In [27]:
dataset = SimulationDataset(config['sims'], config['obs'], config['dat_dir'])
train_loader, test_loader = dataset.get_train_test_loaders(
    train_sims=config['sims'][:-1],  # First 4 sims for training
    test_sim=config['sims'][-1],      # Last sim (TNG_ALL) for testing
    batch_size=config['batch_size']
)

In [28]:
# Infer input dimension from data
sample_X, _, _ = next(iter(train_loader))
config['input_dim'] = sample_X.shape[1]

In [29]:
# Initialize model
model = M.DANN(input_dim=config['input_dim'], 
               num_domains=config['num_domains'], 
               alpha=config['alpha']
        )

In [ ]:
# Train
T.train_dann(
    model, 
    train_loader, 
    test_loader, 
    num_epochs=config['num_epochs'], 
    lr=config['lr'], 
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

In [31]:
# Evaluate
print("\nEvaluating Regression Performance:")
E.evaluate_regression(model, test_loader, 'cpu')

print("\nEvaluating Domain Accuracy:")
E.domain_accuracy(model, train_loader, 'cpu')


Evaluating Regression Performance:
MSE: 0.0955, RMSE: 0.3090, R²: 0.7535

Evaluating Domain Accuracy:
Domain Accuracy: 0.3846


0.38458414554905784